In [1]:
import os
import re

from typing import List

import google.cloud.client

from google.cloud import storage
dev_storage_client = storage.Client()

import ftplib

In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Overall "environment" variables

In [3]:
aria2_flags = '--check-integrity --continue --max-tries=0'

In [4]:
ftp_site = 'ftp.1000genomes.ebi.ac.uk'

ftp_data_root = '/vol1/ftp/data_collections/HGSVC2/working'

In [5]:
gcs_bucket_name = "broad-dsde-methods-long-reads"
gcs_root_prefix = 'datasets/HGSVC2'

In [6]:
gcsfuse_local_dir = 'gs'
gcs_leading_prefix = f'gs://{gcs_bucket_name}/{gcs_root_prefix}'.rstrip('/') + '/'
gcsfuse_local_leading_prefix = f'{gcsfuse_local_dir}/{gcs_root_prefix}'.rstrip('/') + '/'

In [7]:
gcs_dirs = ['20190925_PUR_PacBio_HiFi',
            '20191005_YRI_PacBio_NA19240_HiFi',
            '20191031_CHS_PacBio_HG00512_HiFi',
            '20191031_CHS_PacBio_HG00513_HiFi',
            '20191205_YRI_PacBio_NA19238_HIFI',
            '20191205_YRI_PacBio_NA19239_HIFI',
            '20200108_PacBio_CLR_JAX',
            '20200203_PacBio_CLR_EEE',
            '20200212_PacBio_CLR_Devine',
            '20200722_PUR_PacBio_HG00732_HiFi',
            '20200731_CHS_PacBio_HG00514_CLR_reseq',
            '20200731_CHS_PacBio_HG00514_HiFi_reseq',
            '20210509_UW_HiFi',
            '2021_PacBio_HIFI_JAX'
            ]

In [8]:
known_data_directories_on_ftp = ['20190925_PUR_PacBio_HiFi/',
                                 '20191005_YRI_PacBio_NA19240_HiFi/',
                                 '20191031_CHS_PacBio_HG00512_HiFi/',
                                 '20191031_CHS_PacBio_HG00513_HiFi/',
                                 '20191205_YRI_PacBio_NA19238_HIFI/',
                                 '20191205_YRI_PacBio_NA19239_HIFI/',
                                 '20200108_PacBio_CLR_JAX/',
                                 '20200203_PacBio_CLR_EEE/',
                                 '20200212_PacBio_CLR_Devine/',
                                 '20200722_PUR_PacBio_HG00732_HiFi/',
                                 '20200731_CHS_PacBio_HG00514_CLR_reseq/',
                                 '20200731_CHS_PacBio_HG00514_HiFi_reseq/',
                                 '20210509_UW_HiFi/',
                                 '20210822_UW_EEE_ONT_UL/',
                                 '20210920_ONT_Rebasecalled/',
                                 '20211013_ONT_Rebasecalled/',
                                 '2021_ONT_UltraLong_JAX/',
                                 '2021_PacBio_HIFI_JAX/'
                                 ]

In [9]:
test_dir = gcs_dirs[0]

# Utilities for exploring FTP site

In [10]:
def traverse_ftp_data_dir(configured_ftp: ftplib.FTP, working_dir: str, depth: int=0):
    """
    Return a recursive listing of an ftp server contents, under the current directory.

    :param configured_ftp: ftplib.FTP object with the intended directory to explore
    :param working_dir: current directory being explored
    :param depth: depth into the origin dir
    :return: a recursive dictionary, where
                each key is the name of an entity (a file or a sub-directory), and
                its value is its size in bytes in case of a file, or
                its contents (a dict) in case of a sub-directory.
    """

    if depth > 10:
        raise ValueError("Tree is too deep, I give up.")
    level = {}
    contents = []
    configured_ftp.cwd(working_dir)
    configured_ftp.retrlines('LIST', callback=lambda s: contents.append(s.split()))
    for entry in contents:
        mode = entry[0]
        name = entry[-1]
        if mode.startswith('d'):  # it's a sub-dir (note this is a hack, check this out: https://bit.ly/31zQWLS)
            level[name] = traverse_ftp_data_dir(configured_ftp, name, depth+1)
            configured_ftp.cwd('..')
        else:  # it's a regular file
            level[name] = configured_ftp.size(name)
    return level

def flatten_ftp_dir_tree(nested_dict_tree: dict,
                         configured_ftp: ftplib.FTP,
                         data_dir: str):
    """
    Given the nested dir returned by `traverse_ftp_data_dir`, flatten it to a 1-D
    dict where the key is the path to a file, and value is its size in bytes.

    :param nested_dict_tree: the structure returned by `traverse_ftp_data_dir`
    :param configured_ftp: a ftplib.FTP object where the nested dir was generated on
    :param data_dir: the origin dir where `traverse_ftp_data_dir` was run on
    :return: A list where the entries are absolute paths to each individual file
    """
    flat_file_list = dict()
    for k, v in nested_dict_tree.items():
        if isinstance(v, dict):
            child = flatten_ftp_dir_tree(nested_dict_tree.get(k), configured_ftp = configured_ftp,
                                         data_dir = f'{data_dir}/{k}')
            flat_file_list.update(child)
        else:
            name = f'{configured_ftp.host}{ftp_data_root}/{data_dir}/{k}'
            flat_file_list[name] = v
    return flat_file_list

#### Test

In [11]:
with (ftplib.FTP(ftp_site)) as ftp_socket:
    ftp_socket.login()
    to_explore = ftp_data_root + '/' + test_dir
    print(f"Exploring {to_explore}")
    test_tree = traverse_ftp_data_dir(ftp_socket, working_dir=to_explore)
    test_flat = flatten_ftp_dir_tree(test_tree, ftp_socket, test_dir)
    ftp_socket.quit()

Exploring /vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi


In [12]:
pp.pprint(test_tree)

{   'HG00731_20190925_EEE_m54329U_190528_231241.Q20.fastq.gz': 15898357143,
    'HG00731_20190925_EEE_m54329U_190528_231241.ccs.bam': 16360367615,
    'HG00731_20190925_EEE_m54329U_190528_231241.ccs.bam.bai': 16,
    'HG00731_20190925_EEE_m54329U_190528_231241.ccs.bam.pbi': 18776409,
    'HG00731_20190925_EEE_m54329U_190531_175004.Q20.fastq.gz': 17040893093,
    'HG00731_20190925_EEE_m54329U_190531_175004.ccs.bam': 17552770586,
    'HG00731_20190925_EEE_m54329U_190531_175004.ccs.bam.bai': 16,
    'HG00731_20190925_EEE_m54329U_190531_175004.ccs.bam.pbi': 20713931,
    'HG00731_20190925_EEE_m54329U_190601_235636.Q20.fastq.gz': 16313818362,
    'HG00731_20190925_EEE_m54329U_190601_235636.ccs.bam': 16803785954,
    'HG00731_20190925_EEE_m54329U_190601_235636.ccs.bam.bai': 16,
    'HG00731_20190925_EEE_m54329U_190601_235636.ccs.bam.pbi': 19432762,
    'HG00731_20190925_EEE_m54329U_190606_045526.Q20.fastq.gz': 15568945475,
    'HG00731_20190925_EEE_m54329U_190606_045526.ccs.bam': 16037880166

In [13]:
test_flat

{'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190528_231241.Q20.fastq.gz': 15898357143,
 'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190528_231241.ccs.bam': 16360367615,
 'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190528_231241.ccs.bam.bai': 16,
 'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190528_231241.ccs.bam.pbi': 18776409,
 'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190531_175004.Q20.fastq.gz': 17040893093,
 'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190531_175004.ccs.bam': 17552770586,
 'ftp.1000genomes.ebi.ac.uk/vol1/f

# Utilities for exploring GCS buckets and "folders"

In [14]:
def list_gcs_subdirectories(client: google.cloud.client.Client, bucket_name: str, prefix: str):
    """
    List sub-directories under 'gs://[bucket_name]/[prefix]/'
    Based on https://github.com/googleapis/google-cloud-python/issues/920#issuecomment-653823847
    :param client: GCS client
    :param bucket_name: name of the bucket
    :param prefix: the prefix, or the full path to the directory to list into. Note: don't include leading and ending slash.
    :return: A list of sub-directories under the prefix 'directory' in the provided bucket
    """

    local_prefix = prefix if prefix.endswith('/') else prefix + '/'
    iterator = client.list_blobs(bucket_name, prefix=local_prefix, delimiter='/')
    prefixes = set()
    for page in iterator.pages:
        prefixes.update(page.prefixes)

    return list([s.split('/')[-2] for s in prefixes])

In [15]:
def list_file_base_names(client: google.cloud.client.Client, bucket_name: str, prefix: str):
    """
    Give base name of files under 'gs://[bucket_name]/[prefix]/'
    :param client: GCS client
    :param bucket_name: name of the bucket
    :param prefix: the prefix, or the full path to the directory to list into. Note: don't include leading and ending slash.
    :return: base name of files under the prefix 'directory' in the provided bucket
    """

    full_path = [b.name for b in client.list_blobs(bucket_or_name=bucket_name, prefix=prefix)]
    return [re.sub(f'^{prefix}', '', path).lstrip('/') for path in full_path]

In [16]:
class GcsPath:
    """
    Modeling after GCS storage object, offering simplistic way of
        * checking if the paths exists, and if exists,
        * represents a file or
        * emulates a 'directory'.
    """

    def __init__(self, gs_path: str):

        if not gs_path.startswith("gs://"):
            raise ValueError(f"Provided gs path isn't valid: {gs_path}")

        arr = re.sub("^gs://", '', gs_path).split('/')
        self.bucket = arr[0]
        self.prefix = '/'.join(arr[1:-1])
        self.file = arr[-1]

    def exists(self, client: storage.client.Client) -> bool:
        return self.is_file(client=client) or self.is_emulate_dir(client=client)

    def is_file(self, client: storage.client.Client) -> bool:
        return storage.Blob(bucket=client.bucket(self.bucket), name=f'{self.prefix}/{self.file}').exists(client)

    def is_emulate_dir(self, client: storage.client.Client) -> bool:
        if self.is_file(client=client):
            return False
        return any(True for _ in client.list_blobs(client.bucket(self.bucket), prefix=f'{self.prefix}/{self.file}'))

    def size(self, client: storage.client.Client) -> int:
        blob = storage.Blob(bucket=client.bucket(self.bucket), name=f'{self.prefix}/{self.file}')
        if blob.exists(client=client):
            blob.reload()
            return blob.size
        else:
            return 0

# traverse each dir already on GCS, and check which ones are missing compared to its mirroring folder on FTP

In [17]:
def collect_files_to_transfer(gcs_dir_to_check: str, gcs_client: google.cloud.storage.Client,
                              ftp_server: str, ftp_data_root_dir: str,
                              gcs_bucket_name: str, gcs_root_prefix: str):

    """
    Given a 'folder' on GCS to check data consistency (against a FTP folder), collect which files
       1) have been successfully downloaded,
       2) have been only partially downloaded,
       3) have not been downloaded at all
    The way this is checked is by using file sizes (a compromise compared to MD5 check).
    :param gcs_dir_to_check: 'folder' on GCS to check
    :param gcs_client: GCS python API client
    :param ftp_server: FTP server address
    :param ftp_data_root_dir: root dir holding data, which is supposed to be mirrored by gcs_dir_to_check
    :param gcs_bucket_name: GCS bucket hosting the dir
    :param gcs_root_prefix: root prefix, i.e. 'parent folder' of gcs_dir_to_check, under the hosting bucket
    :return: a tuple-3 of (fully downloaded, incompletely downloaded, not yet downloaded) files
    """

    finished = list()
    incomplete_download = dict()
    not_yet_on_gcs = dict()
    with (ftplib.FTP(ftp_server)) as ftp_socket:
        ftp_socket.login()
        ftp_socket.cwd(ftp_data_root_dir)

        print(f"Traversing test data dir {ftp_data_root_dir}/{gcs_dir_to_check} on FTP server {ftp_server} ...")
        ftp_dir_tree = traverse_ftp_data_dir(ftp_socket, working_dir=gcs_dir_to_check)
        ftp_socket.quit()
        ftp_files_flat = flatten_ftp_dir_tree(ftp_dir_tree, ftp_socket, gcs_dir_to_check)
        print(f"Found {len(ftp_files_flat)} files under {gcs_dir_to_check}")

    print("Matching FTP files with GCS files...")

    for ff, sz in ftp_files_flat.items():
        gs_path = re.sub(f'^{ftp_server}{ftp_data_root}', f'gs://{gcs_bucket_name}/{gcs_root_prefix}', ff)
        to_check = GcsPath(gs_path)
        if to_check.exists(client=gcs_client):
            gcs_sz = to_check.size(client=gcs_client)
            if gcs_sz is None:
                raise ValueError(f"{gs_path} exist on GCS but doesn't have a size...")
            if 0 == gcs_sz:
                not_yet_on_gcs[ff] = gs_path
            elif sz > gcs_sz:
                incomplete_download[ff] = {'full': sz, 'has': gcs_sz, 'gcs_path': gs_path}
            else:
                finished.append(ff)
        else:
            not_yet_on_gcs[ff] = gs_path

    return finished, incomplete_download, not_yet_on_gcs


#### Test

In [18]:
# ok, corrupt, jobs = collect_files_to_transfer(gcs_dir_to_check=test_dir, gcs_client=dev_storage_client,
#                                               ftp_server=ftp_site, ftp_data_root_dir=ftp_data_root,
#                                               gcs_bucket_name=gcs_bucket_name, gcs_root_prefix=gcs_root_prefix)

In [19]:
# corrupt

#### Test one dir where there's sub-dirs

In [20]:
# test_dir = '20200108_PacBio_CLR_JAX'
# ok, corrupt, jobs = collect_files_to_transfer(gcs_dir_to_check=test_dir, gcs_client=dev_storage_client,
#                                               ftp_server=ftp_site, ftp_data_root_dir=ftp_data_root,
#                                               gcs_bucket_name=gcs_bucket_name, gcs_root_prefix=gcs_root_prefix)
# corrupt

# Now real check

In [21]:
missing_files_per_dataset = dict()
for d in gcs_dirs:
    print("====================================================================================================")
    ok, corrupt, fresh = collect_files_to_transfer(gcs_dir_to_check=d, gcs_client=dev_storage_client,
                                                   ftp_server=ftp_site, ftp_data_root_dir=ftp_data_root,
                                                   gcs_bucket_name=gcs_bucket_name, gcs_root_prefix=gcs_root_prefix)
    missing_files_per_dataset[d] = {'corrupt': corrupt, 'fresh': fresh}
    print("====================================================================================================")

Traversing test data dir /vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi on FTP server ftp.1000genomes.ebi.ac.uk ...
Found 52 files under 20190925_PUR_PacBio_HiFi
Matching FTP files with GCS files...
Traversing test data dir /vol1/ftp/data_collections/HGSVC2/working/20191005_YRI_PacBio_NA19240_HiFi on FTP server ftp.1000genomes.ebi.ac.uk ...
Found 26 files under 20191005_YRI_PacBio_NA19240_HiFi
Matching FTP files with GCS files...
Traversing test data dir /vol1/ftp/data_collections/HGSVC2/working/20191031_CHS_PacBio_HG00512_HiFi on FTP server ftp.1000genomes.ebi.ac.uk ...
Found 68 files under 20191031_CHS_PacBio_HG00512_HiFi
Matching FTP files with GCS files...
Traversing test data dir /vol1/ftp/data_collections/HGSVC2/working/20191031_CHS_PacBio_HG00513_HiFi on FTP server ftp.1000genomes.ebi.ac.uk ...
Found 68 files under 20191031_CHS_PacBio_HG00513_HiFi
Matching FTP files with GCS files...
Traversing test data dir /vol1/ftp/data_collections/HGSVC2/working/20191205_

In [22]:
def keep_file(file_full_name: str) -> bool:
    """
    Models a file filter, heavily project-dependent.
    """

    keep = not file_full_name.endswith('.fastq.gz')

    is_meant_for_ccs = any(pat in file_full_name for pat in ['ccs', 'CCS', 'HiFi', 'HIFI'])
    is_pointing_to_non_ccs = any(pat in file_full_name for pat in ['subreads', 'scraps'])
    if is_meant_for_ccs and is_pointing_to_non_ccs:
        return False

    return keep

In [23]:
all_missing_files = {k:v['fresh'] for k,v in missing_files_per_dataset.items()}
all_missing_files = {k: {ik: iv for ik, iv in v.items() if keep_file(iv)} for k, v in all_missing_files.items()}
all_missing_files = {k: v for k, v in all_missing_files.items() if v}
print(f"{sum(len(v) for _, v in all_missing_files.items())} files in total to download, afresh.")
pp.pprint(all_missing_files)

0 files in total to download, afresh.
{}


In [24]:
all_corrupt_files = {k: {ik: iv['gcs_path'] for ik, iv in v['corrupt'].items()} for k,v in missing_files_per_dataset.items()}
all_corrupt_files = {k: {ik: iv for ik, iv in v.items() if keep_file(iv)} for k, v in all_corrupt_files.items()}
all_corrupt_files = {k: v for k, v in all_corrupt_files.items() if v}
print(f"{sum(len(v) for _, v in all_corrupt_files.items())} files in total to download, again.")
pp.pprint(all_corrupt_files)

0 files in total to download, again.
{}


In [25]:
for d, p in all_corrupt_files.items():
    corrupt_files_under_this_dir = missing_files_per_dataset[d].get('corrupt')
    for f in p.keys():
        info = corrupt_files_under_this_dir.get(f)
        n = re.sub(f'{ftp_site}{ftp_data_root}/', '', f)
        print(f"{n}\n\t{info['has']} / {info['full']} bytes")

In [25]:
#### A different file-type filter

In [26]:
def keep_file(file_full_name: str) -> bool:
    """
    One purpose, get rid of incomplete *.fastq.gz
    """

    keep = True
    is_meant_for_ccs = any(pat in file_full_name for pat in ['ccs', 'CCS', 'HiFi', 'HIFI'])
    is_pointing_to_non_ccs = any(pat in file_full_name for pat in ['subreads', 'scraps'])
    if is_meant_for_ccs and is_pointing_to_non_ccs:
        return False

    return keep

In [27]:
all_missing_files = {k:v['fresh'] for k,v in missing_files_per_dataset.items()}
all_missing_files = {k: {ik: iv for ik, iv in v.items() if keep_file(iv)} for k, v in all_missing_files.items()}
all_missing_files = {k: v for k, v in all_missing_files.items() if v}
print(f"{sum(len(v) for _, v in all_missing_files.items())} files in total to download, afresh.")
pp.pprint(all_missing_files)

3 files in total to download, afresh.
{   '20191031_CHS_PacBio_HG00513_HiFi': {   'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20191031_CHS_PacBio_HG00513_HiFi/HG00513_PacBio_Sequel2_CCS__EDEVI_20191004_S64018_PL100139348-1_D01.ccs.fastq.gz': 'gs://broad-dsde-methods-long-reads/datasets/HGSVC2/20191031_CHS_PacBio_HG00513_HiFi/HG00513_PacBio_Sequel2_CCS__EDEVI_20191004_S64018_PL100139348-1_D01.ccs.fastq.gz',
                                            'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20191031_CHS_PacBio_HG00513_HiFi/HG00513_PacBio_Sequel2_CCS__EDEVI_20191007_S64049_PL100139348-2_C01.ccs.fastq.gz': 'gs://broad-dsde-methods-long-reads/datasets/HGSVC2/20191031_CHS_PacBio_HG00513_HiFi/HG00513_PacBio_Sequel2_CCS__EDEVI_20191007_S64049_PL100139348-2_C01.ccs.fastq.gz',
                                            'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20191031_CHS_PacBio_HG00513_HiFi/HG00513_PacBio_Sequel2_CC

In [28]:
all_corrupt_files = {k: {ik: iv['gcs_path'] for ik, iv in v['corrupt'].items()} for k,v in missing_files_per_dataset.items()}
all_corrupt_files = {k: {ik: iv for ik, iv in v.items() if keep_file(iv)} for k, v in all_corrupt_files.items()}
all_corrupt_files = {k: v for k, v in all_corrupt_files.items() if v}
print(f"{sum(len(v) for _, v in all_corrupt_files.items())} files in total to download, again.")
pp.pprint(all_corrupt_files)

17 files in total to download, again.
{   '20190925_PUR_PacBio_HiFi': {   'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190528_231241.Q20.fastq.gz': 'gs://broad-dsde-methods-long-reads/datasets/HGSVC2/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190528_231241.Q20.fastq.gz',
                                    'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190531_175004.Q20.fastq.gz': 'gs://broad-dsde-methods-long-reads/datasets/HGSVC2/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190531_175004.Q20.fastq.gz',
                                    'ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190601_235636.Q20.fastq.gz': 'gs://broad-dsde-methods-long-reads/datasets/HGSVC2/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190601_235636.Q20.fastq.g

In [29]:
for d, p in all_corrupt_files.items():
    corrupt_files_under_this_dir = missing_files_per_dataset[d].get('corrupt')
    for f in p.keys():
        info = corrupt_files_under_this_dir.get(f)
        n = re.sub(f'{ftp_site}{ftp_data_root}/', '', f)
        print(f"{n}\n\t{info['has']} / {info['full']} bytes")

20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190528_231241.Q20.fastq.gz
	14392681785 / 15898357143 bytes
20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190531_175004.Q20.fastq.gz
	7135434360 / 17040893093 bytes
20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190601_235636.Q20.fastq.gz
	7121111496 / 16313818362 bytes
20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190606_045526.Q20.fastq.gz
	7164008468 / 15568945475 bytes
20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190906_205127.Q20.fastq.gz
	7121648588 / 16591637237 bytes
20190925_PUR_PacBio_HiFi/HG00733_20190925_EEE_m54329U_190615_010947.Q20.fastq.gz
	7103838920 / 9966703426 bytes
20190925_PUR_PacBio_HiFi/HG00733_20190925_EEE_m54329U_190617_231905.Q20.fastq.gz
	7116162640 / 9486694384 bytes
20190925_PUR_PacBio_HiFi/HG00733_20190925_EEE_m54329U_190619_052546.Q20.fastq.gz
	7197075084 / 7594807936 bytes
20190925_PUR_PacBio_HiFi/HG00733_20190925_EEE_m54329U_190629_180018.Q20.fastq.gz
	7099348620 / 126

In [30]:
if 0 < len(all_corrupt_files):
    # remove corrupt files from cloud first:
    # !!!!!!!!!! MAKE SURE THEY ARE NOT IN THE PROCESS OF BEING DOWNLOADED !!!!!!!!!!
    for k, v in all_corrupt_files.items():
        for _, gs_path in v.items():
            cmd = f'gsutil rm {gs_path}'
            os.system(cmd)

Removing gs://broad-dsde-methods-long-reads/datasets/HGSVC2/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190528_231241.Q20.fastq.gz...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Removing gs://broad-dsde-methods-long-reads/datasets/HGSVC2/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190531_175004.Q20.fastq.gz...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Removing gs://broad-dsde-methods-long-reads/datasets/HGSVC2/20190925_PUR_PacBio_HiFi/HG00731_20190925_EEE_m54329U_190601_235636.Q20.fastq.gz...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Removing gs://broad-dsde-methods-long-reads/datasets/HGSVC2/20190925_PUR_PacBio

#### print out file downloading commands

In [28]:
if 0 < len(all_missing_files):
    for k,v in all_missing_files.items():
        for ftp_path, gs_path in v.items():

            local_relative_path = re.sub(gcs_leading_prefix, '', gs_path)
            local_prefix = gcsfuse_local_leading_prefix + '/'.join(local_relative_path.split('/')[:-1]) + '/'
            log_file = '__'.join(ftp_path.split('/')[6:]) + ".aria2-log"
            output_file_name = local_relative_path.split('/')[-1]
            cmd = f'aria2c \\\n {aria2_flags} \\\n  --dir={local_prefix} \\\n  --log={log_file} \\\n  --out={output_file_name} \\\n  ftp://{ftp_path}'
            print(cmd)
            with open(f'/Users/shuang/Desktop/hgsvc2_more_downloads/aria2/{output_file_name}.download.sh', 'w') as out:
                out.write('#!/bin/bash\n\nset -eu\n\n')
                out.write(cmd)
                out.write('\n')

aria2c \
 --check-integrity --continue --max-tries=0 \
  --dir=gs/datasets/HGSVC2/20200108_PacBio_CLR_JAX/20200124_JAX_CLR_additional_samples/ \
  --log=20200108_PacBio_CLR_JAX__20200124_JAX_CLR_additional_samples__HG03732_20200108_CLEE_m54336U_191230_150456.subreads.bam.wget-log \
  --out=HG03732_20200108_CLEE_m54336U_191230_150456.subreads.bam \
  ftP://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20200108_PacBio_CLR_JAX/20200124_JAX_CLR_additional_samples/HG03732_20200108_CLEE_m54336U_191230_150456.subreads.bam
aria2c \
 --check-integrity --continue --max-tries=0 \
  --dir=gs/datasets/HGSVC2/20200108_PacBio_CLR_JAX/20200124_JAX_CLR_additional_samples/ \
  --log=20200108_PacBio_CLR_JAX__20200124_JAX_CLR_additional_samples__HG03732_20200116_CLEE_m54336U_200112_083928.subreads.bam.wget-log \
  --out=HG03732_20200116_CLEE_m54336U_200112_083928.subreads.bam \
  ftP://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/HGSVC2/working/20200108_PacBio_CLR_JAX/20200124_

In [ ]:
if 0 < len(all_corrupt_files):
    # remove corrupt files from cloud first:
    # !!!!!!!!!! MAKE SURE THEY ARE NOT IN THE PROCESS OF BEING DOWNLOADED !!!!!!!!!!
    for k, v in all_corrupt_files.items():
        for _, gs_path in v.items():
            cmd = f'gsutil rm {gs_path}'
            os.system(cmd)
    for k,v in all_corrupt_files.items():
        for ftp_path, gs_path in v.items():

            local_relative_path = re.sub(gcs_leading_prefix, '', gs_path)
            local_prefix = gcsfuse_local_leading_prefix + '/'.join(local_relative_path.split('/')[:-1]) + '/'
            log_file = '__'.join(ftp_path.split('/')[6:]) + ".wget-log"
            output_file_name = local_relative_path.split('/')[-1]
            cmd = f'aria2c \\\n {aria2_flags} \\\n  --dir={local_prefix} \\\n  --log={log_file} \\\n  --out={output_file_name} \\\n  ftp://{ftp_path}'
            print(cmd)
            with open(f'/Users/shuang/Desktop/hgsvc2_more_downloads/aria2/{output_file_name}.download.sh', 'w') as out:
                out.write('#!/bin/bash\n\nset -eu\n\n')
                out.write(cmd)
                out.write('\n')